### Data Science Capstone Course

In [0]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

### Part 1: Web scraping 

In [0]:
#lấy data từ wiki 
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [0]:
#chuyển data thành đối tượng object của gói beautifulsoup 
soup = BeautifulSoup(data,'html.parser')

In [0]:
#Tìm tất cả các [hàng (dòng) thẻ 'tr':table row] - thẻ 'td':table data: là các cell
rows = soup.find('table').find_all('tr')
#rows

In [9]:
row = rows[5]#truy xuất các dòng
cells=row.find_all('td')
cells[2]#truy xuất các cell

<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td>

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. 
* Ignore cells with a borough that is Not assigned.

In [0]:
postalCodeList = []
boroughList = []
neighborhoodList = []
for row in rows:
  cells =  row.find_all('td')
  if(len(cells)>0):
    postalCodeList.append(cells[0].text)
    boroughList.append(cells[1].text)
    neighborhoodList.append(cells[2].text.rstrip('\n')) # tranh dòng mới gần cell

In [15]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})
toronto_df.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [0]:
#toronto_df.drop("Not assigned", axis=0)=>giữ lại những dòng trong bảng mà ở cột Borough != "Not assigned"
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned"]
toronto_df_drop.reset_index(drop=True)

In [0]:
#nhóm 2 cột đầu
toronto_df_drop_groupby = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False)


In [0]:
#gôp các giá trị ở cột Neighborhood đã nhóm 2 cột đầu
toronto_df_drop_groupby = toronto_df_drop_groupby.agg(lambda x: ", ".join(x))


In [21]:
toronto_df_drop_groupby.shape

(103, 3)

In [22]:
# mỗi Neighborhood="Not assigned", gán lại giá trị ở cột Borough tương ứng
for index, row in toronto_df_drop_groupby.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_drop_groupby.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [23]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
check_list_df = pd.DataFrame(columns=column_names)
for postcode in ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]:
  check_list_df = check_list_df.append(toronto_df_drop_groupby[toronto_df_drop_groupby.PostalCode==postcode], ignore_index=True)
check_list_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens, Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale, Wexford"
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


In [0]:
na_neigh_rows = toronto_df_drop_groupby.Neighborhood == toronto_df_drop_groupby.Borough

* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [0]:
toronto_df_drop_groupby[na_neigh_rows]

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [0]:
toronto_df_drop_groupby.shape

(103, 3)